# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,luderitz,-26.6481,15.1594,18.97,84,3,3.04,NaN,1706302174
1,1,thompson,55.7435,-97.8558,-4.91,93,100,4.63,CA,1706302174
2,2,puerto ayora,-0.7393,-90.3518,29.98,90,61,3.58,EC,1706302174
3,3,east london,-33.0153,27.9116,24.32,69,9,1.33,ZA,1706302174
4,4,xai-xai,-25.0519,33.6442,22.54,84,0,3.56,MZ,1706302175


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:
%%capture --no-display

# Configure the map plot
all_cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.7
)

# Display the map
all_cities_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Drop any rows with null values
city_data_cleaned = city_data_df.dropna(axis=0, how="any", inplace=False)

# Narrow down cities that fit criteria
# My Criteria
my_high_temp = 32
my_low_temp = 20
my_humid = 35
my_clouds = 20

my_cities = city_data_cleaned.loc[(city_data_cleaned["Max Temp"] >= my_low_temp) & 
                                  (city_data_cleaned["Max Temp"] < my_high_temp) &
                                  (city_data_cleaned["Humidity"] < my_humid) & 
                                  (city_data_cleaned["Cloudiness"] < my_clouds),:]

# Display sample data
my_cities.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,diego de almagro,-26.3667,-70.0500,29.15,27,0,6.04,CL,1706302094
52,52,hohenau,-27.0833,-55.7500,29.65,32,7,4.34,PY,1706302185
63,63,touba,8.2833,-7.6833,26.08,17,0,0.91,CI,1706302187
91,91,smara,26.7384,-11.6719,20.43,18,0,5.17,EH,1706302193
107,107,punta arenas,-53.1500,-70.9167,22.01,32,0,5.14,CL,1706302164
125,125,leon,21.1167,-101.6667,25.66,15,0,6.80,MX,1706302133
214,214,tougue,11.4667,-11.6000,21.61,15,0,1.11,GN,1706302223
284,284,chignahuapan,19.8333,-98.0333,22.07,27,2,2.28,MX,1706302242
360,360,puerto deseado,-47.7503,-65.8938,29.74,24,8,12.52,AR,1706302263
410,410,cove,7.2210,2.3402,27.61,22,17,1.18,BJ,1706302275


### Step 3: Create a new DataFrame called `hotel_df`.

In [50]:
hotel_df = my_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy().reset_index(drop=True)
hotel_df


,City,Country,Lat,Lng,Humidity
0,diego de almagro,CL,-26.3667,-70.0500,27
1,hohenau,PY,-27.0833,-55.7500,32
2,touba,CI,8.2833,-7.6833,17
3,smara,EH,26.7384,-11.6719,18
4,punta arenas,CL,-53.1500,-70.9167,32
5,leon,MX,21.1167,-101.6667,15
6,tougue,GN,11.4667,-11.6000,15
7,chignahuapan,MX,19.8333,-98.0333,27
8,puerto deseado,AR,-47.7503,-65.8938,24
9,cove,BJ,7.2210,2.3402,22


In [51]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy().reset_index(drop=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,diego de almagro,CL,-26.3667,-70.0500,27,
1,hohenau,PY,-27.0833,-55.7500,32,
2,touba,CI,8.2833,-7.6833,17,
3,smara,EH,26.7384,-11.6719,18,
4,punta arenas,CL,-53.1500,-70.9167,32,
5,leon,MX,21.1167,-101.6667,15,
6,tougue,GN,11.4667,-11.6000,15,
7,chignahuapan,MX,19.8333,-98.0333,27,
8,puerto deseado,AR,-47.7503,-65.8938,24,
9,cove,BJ,7.2210,2.3402,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [53]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit":1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL 
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty & Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()


    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
diego de almagro - nearest hotel: Residencial Hambyll
hohenau - nearest hotel: Posada Don Paraguay
touba - nearest hotel: Hôtel de l'amitié vrai
smara - nearest hotel: Hôtel Amine فندق الأمين
punta arenas - nearest hotel: Hotel Lacolet
leon - nearest hotel: Hotsson
tougue - nearest hotel: No hotel found
chignahuapan - nearest hotel: SYMER
puerto deseado - nearest hotel: Los Olmos
cove - nearest hotel: TG Hôtel Covè
tazacorte - nearest hotel: App Leyma
botro - nearest hotel: No hotel found
fada - nearest hotel: No hotel found
alo - nearest hotel: No hotel found
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
0,diego de almagro,CL,-26.3667,-70.0500,27,Residencial Hambyll
1,hohenau,PY,-27.0833,-55.7500,32,Posada Don Paraguay
2,touba,CI,8.2833,-7.6833,17,Hôtel de l'amitié vrai
3,smara,EH,26.7384,-11.6719,18,Hôtel Amine فندق الأمين
4,punta arenas,CL,-53.1500,-70.9167,32,Hotel Lacolet
5,leon,MX,21.1167,-101.6667,15,Hotsson
6,tougue,GN,11.4667,-11.6000,15,No hotel found
7,chignahuapan,MX,19.8333,-98.0333,27,SYMER
8,puerto deseado,AR,-47.7503,-65.8938,24,Los Olmos
9,cove,BJ,7.2210,2.3402,22,TG Hôtel Covè


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
%%capture --no-display

# Configure the map plot

my_hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    hover_cols=["Hotel Name", "Country"],
    alpha=.7       
)

# Display the map
my_hotels_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)